In [2]:
%reload_ext autotime
import geopandas as gpd
import pandas as pd
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from coastsat import SDS_transects

/home/medcoast/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5d11eb2060, execution_count=2 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5d11eb0650, raw_cell="%reload_ext autotime
import geopandas as gpd
impor.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [3]:
# Transects, origin is landward. Has beach_slope
transects = gpd.read_file("transects_extended.geojson")
transects.set_index("id", inplace=True)
transects

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccf47ef00, raw_cell="# Transects, origin is landward. Has beach_slope
t.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W1sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2540-0000,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (8.78067 38.91452, 8.77915 38.91209)"
sar2540-0001,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (8.78158 38.91412, 8.77927 38.9121)"
sar2540-0002,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (8.78201 38.91382, 8.7797 38.9118)"


Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccf47ef30, execution_count=3 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccf47ef00, raw_cell="# Transects, origin is landward. Has beach_slope
t.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W1sdnNjb2RlLXJlbW90ZQ%3D%3D> result=              site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.008900

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [4]:
vos_files = pd.Series(
    sorted(glob("shoreline_data_run6/*/time_series_tidally_corrected.csv"))
)
vos_files = vos_files[~vos_files.str.contains("nzd")]
vos_files

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccf4d2a50, raw_cell="vos_files = pd.Series(
    sorted(glob("shoreline_.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W2sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
3064    shoreline_data_run6/usa_HI_0051/time_series_ti...
3065    shoreline_data_run6/usa_HI_0052/time_series_ti...
3066    shoreline_data_run6/usa_HI_0053/time_series_ti...
3067    shoreline_data_run6/usa_HI_0054/time_series_ti...
3068    shoreline_data_run6/usa_HI_0057/time_series_ti...
Length: 2509, dtype: object

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5cd7206c30, execution_count=4 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccf4d2a50, raw_cell="vos_files = pd.Series(
    sorted(glob("shoreline_.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W2sdnNjb2RlLXJlbW90ZQ%3D%3D> result=0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
3064    shoreline_data_run6/usa_HI_0051/time_series_ti...
3065    shoreline_data_run6/usa_HI_0052/time_series_ti...

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [5]:
my_nz_files = pd.Series(
    sorted(glob("data/*/transect_time_series_tidally_corrected.csv"))
)
my_nz_files

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccaa28440, raw_cell="my_nz_files = pd.Series(
    sorted(glob("data/*/t.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W3sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

0      data/nzd0001/transect_time_series_tidally_corr...
1      data/nzd0002/transect_time_series_tidally_corr...
2      data/nzd0003/transect_time_series_tidally_corr...
3      data/nzd0004/transect_time_series_tidally_corr...
4      data/nzd0005/transect_time_series_tidally_corr...
                             ...                        
557    data/nzd0559/transect_time_series_tidally_corr...
558    data/nzd0560/transect_time_series_tidally_corr...
559    data/nzd0561/transect_time_series_tidally_corr...
560    data/nzd0562/transect_time_series_tidally_corr...
561    data/nzd0563/transect_time_series_tidally_corr...
Length: 562, dtype: object

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa283e0, execution_count=5 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccaa28440, raw_cell="my_nz_files = pd.Series(
    sorted(glob("data/*/t.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W3sdnNjb2RlLXJlbW90ZQ%3D%3D> result=0      data/nzd0001/transect_time_series_tidally_corr...
1      data/nzd0002/transect_time_series_tidally_corr...
2      data/nzd0003/transect_time_series_tidally_corr...
3      data/nzd0004/transect_time_series_tidally_corr...
4      data/nzd0005/transect_time_series_tidally_corr...
                             ...                        
557    data/nzd0559/transect_time_series_tidally_corr...
558    data/nzd0560/transect_time_series_tidally_corr...
559    

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [6]:
sar_files = pd.Series(sorted(glob("data/sar*/transect_time_series.csv")))
sar_files

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5cca99e120, raw_cell="sar_files = pd.Series(sorted(glob("data/sar*/trans.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W4sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

0       data/sar0001/transect_time_series.csv
1       data/sar0002/transect_time_series.csv
2       data/sar0003/transect_time_series.csv
3       data/sar0004/transect_time_series.csv
4       data/sar0005/transect_time_series.csv
                        ...                  
2535    data/sar2536/transect_time_series.csv
2536    data/sar2537/transect_time_series.csv
2537    data/sar2538/transect_time_series.csv
2538    data/sar2539/transect_time_series.csv
2539    data/sar2540/transect_time_series.csv
Length: 2540, dtype: object

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5cca99e0f0, execution_count=6 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5cca99e120, raw_cell="sar_files = pd.Series(sorted(glob("data/sar*/trans.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W4sdnNjb2RlLXJlbW90ZQ%3D%3D> result=0       data/sar0001/transect_time_series.csv
1       data/sar0002/transect_time_series.csv
2       data/sar0003/transect_time_series.csv
3       data/sar0004/transect_time_series.csv
4       data/sar0005/transect_time_series.csv
                        ...                  
2535    data/sar2536/transect_time_series.csv
2536    data/sar2537/transect_time_series.csv
2537    data/sar2538/transect_time_series.csv
2538    data/sar2539/transect_time_series.csv
253

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [7]:
files = pd.concat([vos_files, my_nz_files, sar_files])
files

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccaa2a1e0, raw_cell="files = pd.concat([vos_files, my_nz_files, sar_fil.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W5sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
2535                data/sar2536/transect_time_series.csv
2536                data/sar2537/transect_time_series.csv
2537                data/sar2538/transect_time_series.csv
2538                data/sar2539/transect_time_series.csv
2539                data/sar2540/transect_time_series.csv
Length: 5611, dtype: object

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa2b410, execution_count=7 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccaa2a1e0, raw_cell="files = pd.concat([vos_files, my_nz_files, sar_fil.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W5sdnNjb2RlLXJlbW90ZQ%3D%3D> result=0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
2535                data/sar2536/transect_time_series.csv
2536                data/sar2537/transect_time_series.csv

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [8]:
"""f = sar_files.iloc[0]
despiked_filename = f.replace(".csv", "_despiked.csv")
df = pd.read_csv(despiked_filename)
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
import matplotlib.pyplot as plt
for transect_id in df.columns:
  sample = df[transect_id]
  ax = sample.plot(figsize=(10,10), style=".-", label="raw data")
  sample.rolling(10, min_periods=1).mean().plot(ax=ax, label="rolling mean")
  ax.legend()
  plt.show()"""

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccaa282f0, raw_cell=""""f = sar_files.iloc[0]
despiked_filename = f.rep.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W6sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

'f = sar_files.iloc[0]\ndespiked_filename = f.replace(".csv", "_despiked.csv")\ndf = pd.read_csv(despiked_filename)\ndf.dates = pd.to_datetime(df.dates)\ndf.set_index("dates", inplace=True)\nimport matplotlib.pyplot as plt\nfor transect_id in df.columns:\n  sample = df[transect_id]\n  ax = sample.plot(figsize=(10,10), style=".-", label="raw data")\n  sample.rolling(10, min_periods=1).mean().plot(ax=ax, label="rolling mean")\n  ax.legend()\n  plt.show()'

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa28200, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccaa282f0, raw_cell=""""f = sar_files.iloc[0]
despiked_filename = f.rep.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#W6sdnNjb2RlLXJlbW90ZQ%3D%3D> result='f = sar_files.iloc[0]\ndespiked_filename = f.replace(".csv", "_despiked.csv")\ndf = pd.read_csv(despiked_filename)\ndf.dates = pd.to_datetime(df.dates)\ndf.set_index("dates", inplace=True)\nimport matplotlib.pyplot as plt\nfor transect_id in df.columns:\n  sample = df[transect_id]\n  ax = sample.plot(figsize=(10,10), style=".-", label="raw data")\n  sample.rolling(10, min_periods=1).mean().plot(ax=ax, label="rolling mean")\n  ax.legend()\n  plt.show()'>,),kw

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [ ]:
#trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")
#len(trends)
def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)


def get_trends(f):
    df = pd.read_csv(f)
    try:
        df.dates = pd.to_datetime(df.dates)
    except:
        print(f)
    if "sar" in f:
        despiked_filename = f.replace(".csv", "_despiked.csv")
        try:
            raise
            df = pd.read_csv(despiked_filename)
            df.dates = pd.to_datetime(df.dates)
        except:
            df.dates = pd.to_datetime(df.dates)
            df.set_index("dates", inplace=True)
            satname = df.satname
            df = df.drop(columns="satname").apply(despike, axis=0)
            df["satname"] = satname
            df.reset_index(names="dates", inplace=True)
            df.to_csv(f.replace(".csv", "_despiked.csv"), index=False)
    df.index = (df.dates - df.dates.min()).dt.days / 365.25
    df.drop(columns=["dates", "satname", "Unnamed: 0"], inplace=True, errors="ignore")
    trends = []
    for transect_id in df.columns:
        sub_df = df[transect_id].dropna()
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1, 1)
        y = sub_df
        linear_model = LinearRegression().fit(x, y)
        pred = linear_model.predict(x)
        trends.append(
            {
                "transect_id": transect_id,
                "trend": linear_model.coef_[0],
                "intercept": linear_model.intercept_,
                "n_points": len(df[transect_id]),
                "n_points_nonan": len(sub_df),
                "r2_score": r2_score(y, pred),
                "mae": mean_absolute_error(y, pred),
                "mse": mean_squared_error(y, pred),
                "rmse": root_mean_squared_error(y, pred),
            }
        )
    return pd.DataFrame(trends)


#trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")
#len(trends)
trends=get_trends(sar_files.iloc[-1]).set_index("transect_id")

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccb560500, raw_cell="#trends = pd.concat(process_map(get_trends, files).." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X10sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

**Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa737d0, execution_count=13 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccb560500, raw_cell="#trends = pd.concat(process_map(get_trends, files).." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X10sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [ ]:
trends

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ce13ec110, raw_cell="trends" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X24sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
sar2540-0000,0.230656,208.319661,1261,1068,0.014379,19.726461,544.099943,23.325950
sar2540-0001,-0.179234,179.467747,1261,1218,0.042395,8.166520,107.185533,10.353045
sar2540-0002,-0.182292,174.938721,1261,1226,0.045167,8.070813,103.598035,10.178312
sar2540-0003,-0.156003,169.587577,1261,1204,0.037256,7.475358,92.953779,9.641254
sar2540-0004,-0.150395,175.497527,1261,1189,0.033152,8.075896,98.327260,9.916010


Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ce13ecf20, execution_count=14 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ce13ec110, raw_cell="trends" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X24sdnNjb2RlLXJlbW90ZQ%3D%3D> result=                 trend   intercept  n_points  n_points_nonan  r2_score  \
transect_id                                                              
sar2540-0000  0.230656  208.319661      1261            1068  0.014379   
sar2540-0001 -0.179234  179.467747      1261            1218  0.042395   
sar2540-0002 -0.182292  174.938721      1261            1226  0.045167   
sar2540-0003 -0.156003  169.587577      1261            1204  0.037256   
sar2540-0004 -0.150395  175.497527      1261            1189  0

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [ ]:
trends[trends.n_points_nonan > 10].sort_values("r2_score")

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
sar1247-0000,-4.976915e-07,213.492243,677,390,4.544143e-13,7.008953,79.269336,8.903333
aus0213-0000,-2.941492e-06,211.475107,543,487,8.622880e-12,7.797467,94.136738,9.702409
chi0406-0028,5.363548e-06,183.818769,456,396,1.916567e-11,6.755653,85.309469,9.236312
mex0157-0034,-8.100513e-06,225.225194,496,413,3.072342e-11,10.834152,183.654195,13.551907
usa_CA_0052-0040,-9.913390e-06,226.224974,1340,971,5.089262e-11,10.782984,177.958484,13.340108
...,...,...,...,...,...,...,...,...
sar0811-0000,-7.032405e+00,362.639306,557,485,9.855151e-01,7.916487,96.965647,9.847114
sar0812-0006,-6.572763e+00,355.297751,499,407,9.858723e-01,7.335879,83.388304,9.131720
sar0811-0004,-7.009909e+00,390.591254,557,485,9.876367e-01,6.645366,69.838904,8.356967


In [ ]:
trends[trends.index.str.startswith("sar")].describe()

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccb89e3c0, raw_cell="trends[trends.index.str.startswith("sar")].describ.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X12sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.000000
mean,-0.087454,181.562247,1261.0,1181.000000,0.034470,10.303009,189.232910,12.682914
std,0.178376,15.365471,0.0,64.722485,0.012149,5.275044,198.449756,5.955732
min,-0.182292,169.587577,1261.0,1068.000000,0.014379,7.475358,92.953779,9.641254
25%,-0.179234,174.938721,1261.0,1189.000000,0.033152,8.070813,98.327260,9.916010
50%,-0.156003,175.497527,1261.0,1204.000000,0.037256,8.075896,103.598035,10.178312
75%,-0.150395,179.467747,1261.0,1218.000000,0.042395,8.166520,107.185533,10.353045
max,0.230656,208.319661,1261.0,1226.000000,0.045167,19.726461,544.099943,23.325950


Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccb89ebd0, execution_count=15 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccb89e3c0, raw_cell="trends[trends.index.str.startswith("sar")].describ.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X12sdnNjb2RlLXJlbW90ZQ%3D%3D> result=          trend   intercept  n_points  n_points_nonan  r2_score        mae  \
count  5.000000    5.000000       5.0        5.000000  5.000000   5.000000   
mean  -0.087454  181.562247    1261.0     1181.000000  0.034470  10.303009   
std    0.178376   15.365471       0.0       64.722485  0.012149   5.275044   
min   -0.182292  169.587577    1261.0     1068.000000  0.014379   7.475358   
25%   -0.179234  174.938721    1261.0     1189.000000  0.033152   8.070

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [ ]:
trends.describe()

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ce13ec290, raw_cell="trends.describe()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X13sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.000000
mean,-0.087454,181.562247,1261.0,1181.000000,0.034470,10.303009,189.232910,12.682914
std,0.178376,15.365471,0.0,64.722485,0.012149,5.275044,198.449756,5.955732
min,-0.182292,169.587577,1261.0,1068.000000,0.014379,7.475358,92.953779,9.641254
25%,-0.179234,174.938721,1261.0,1189.000000,0.033152,8.070813,98.327260,9.916010
50%,-0.156003,175.497527,1261.0,1204.000000,0.037256,8.075896,103.598035,10.178312
75%,-0.150395,179.467747,1261.0,1218.000000,0.042395,8.166520,107.185533,10.353045
max,0.230656,208.319661,1261.0,1226.000000,0.045167,19.726461,544.099943,23.325950


Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa737d0, execution_count=16 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ce13ec290, raw_cell="trends.describe()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X13sdnNjb2RlLXJlbW90ZQ%3D%3D> result=          trend   intercept  n_points  n_points_nonan  r2_score        mae  \
count  5.000000    5.000000       5.0        5.000000  5.000000   5.000000   
mean  -0.087454  181.562247    1261.0     1181.000000  0.034470  10.303009   
std    0.178376   15.365471       0.0       64.722485  0.012149   5.275044   
min   -0.182292  169.587577    1261.0     1068.000000  0.014379   7.475358   
25%   -0.179234  174.938721    1261.0     1189.000000  0.033152   8.070813   
50%   -0.156003  175.497527 

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [17]:
(transects.trend - trends.trend).describe()

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ce13effb0, raw_cell="(transects.trend - trends.trend).describe()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X14sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: trend, dtype: float64

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccb89e9c0, execution_count=17 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ce13effb0, raw_cell="(transects.trend - trends.trend).describe()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X14sdnNjb2RlLXJlbW90ZQ%3D%3D> result=count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: trend, dtype: float64>,),kwargs {}:


TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [ ]:
trends["ERODIBILITY"]="High"

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ce13ef140, raw_cell="trends["ERODIBILITY"]="High"" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X25sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa737d0, execution_count=22 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ce13ef140, raw_cell="trends["ERODIBILITY"]="High"" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X25sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [ ]:
transects.update(trends.drop_duplicates())

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ce107ff80, raw_cell="transects.update(trends.drop_duplicates())" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X15sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ce107fa70, execution_count=23 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ce107ff80, raw_cell="transects.update(trends.drop_duplicates())" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X15sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [24]:
transects

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ce13ee900, raw_cell="transects" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X16sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2540-0000,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,0.230656,1261.0,1068.0,0.014379,19.726461,544.099943,23.325950,208.319661,High,"LINESTRING (8.78067 38.91452, 8.77915 38.91209)"
sar2540-0001,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.179234,1261.0,1218.0,0.042395,8.166520,107.185533,10.353045,179.467747,High,"LINESTRING (8.78158 38.91412, 8.77927 38.9121)"
sar2540-0002,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.182292,1261.0,1226.0,0.045167,8.070813,103.598035,10.178312,174.938721,High,"LINESTRING (8.78201 38.91382, 8.7797 38.9118)"


Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa737d0, execution_count=24 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ce13ee900, raw_cell="transects" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X16sdnNjb2RlLXJlbW90ZQ%3D%3D> result=              site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.008900        0.055   
aus0001-0004  aus0001   

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [25]:
trends.columns, transects.columns, trends.columns.isin(transects.columns)

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccb89fe90, raw_cell="trends.columns, transects.columns, trends.columns..." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X20sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
        'mse', 'rmse', 'ERODIBILITY'],
       dtype='object'),
 Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
        'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
        'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'ERODIBILITY',
        'geometry'],
       dtype='object'),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True]))

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccb89c8f0, execution_count=25 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccb89fe90, raw_cell="trends.columns, transects.columns, trends.columns..." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X20sdnNjb2RlLXJlbW90ZQ%3D%3D> result=(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
       'mse', 'rmse', 'ERODIBILITY'],
      dtype='object'), Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
       'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
       'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'ERODIBILITY',
       'geometry'],
      dtype='object'), array([ True,  True,  True,  True,  True,  True,  True,  True,  True

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [26]:
transects = transects.join(trends.loc[:, ~trends.columns.isin(transects.columns)])
transects

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ce13ec8f0, raw_cell="transects = transects.join(trends.loc[:, ~trends.c.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X21sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2540-0000,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,0.230656,1261.0,1068.0,0.014379,19.726461,544.099943,23.325950,208.319661,High,"LINESTRING (8.78067 38.91452, 8.77915 38.91209)"
sar2540-0001,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.179234,1261.0,1218.0,0.042395,8.166520,107.185533,10.353045,179.467747,High,"LINESTRING (8.78158 38.91412, 8.77927 38.9121)"
sar2540-0002,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.182292,1261.0,1226.0,0.045167,8.070813,103.598035,10.178312,174.938721,High,"LINESTRING (8.78201 38.91382, 8.7797 38.9118)"


Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa737d0, execution_count=26 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ce13ec8f0, raw_cell="transects = transects.join(trends.loc[:, ~trends.c.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X21sdnNjb2RlLXJlbW90ZQ%3D%3D> result=              site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.0089

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [27]:
transects[transects.site_id.str.startswith("sar") & ~transects.trend.isna()]

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ccb89d0d0, raw_cell="transects[transects.site_id.str.startswith("sar") .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X22sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
sar0001-0000,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.068364,664.0,508.0,0.000113,52.578021,6137.692249,78.343425,124.161754,Medium,"LINESTRING (8.40852 38.86175, 8.40882 38.86535)"
sar0001-0001,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.197266,664.0,498.0,0.002798,19.969642,2010.059995,44.833693,197.183041,Medium,"LINESTRING (8.4084 38.86162, 8.41092 38.86464)"
sar0001-0002,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.009191,664.0,545.0,0.000059,11.768691,215.362959,14.675250,206.410997,Medium,"LINESTRING (8.40893 38.86153, 8.41236 38.86393)"
sar0001-0003,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.040081,664.0,597.0,0.002059,8.475977,112.442187,10.603876,239.306868,Medium,"LINESTRING (8.40904 38.86129, 8.41333 38.8626)"
sar0001-0004,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.009671,664.0,603.0,0.000141,7.011139,96.998364,9.848775,258.388646,Medium,"LINESTRING (8.40904 38.86176, 8.4133 38.86037)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2540-0000,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,0.230656,1261.0,1068.0,0.014379,19.726461,544.099943,23.325950,208.319661,High,"LINESTRING (8.78067 38.91452, 8.77915 38.91209)"
sar2540-0001,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.179234,1261.0,1218.0,0.042395,8.166520,107.185533,10.353045,179.467747,High,"LINESTRING (8.78158 38.91412, 8.77927 38.9121)"
sar2540-0002,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.182292,1261.0,1226.0,0.045167,8.070813,103.598035,10.178312,174.938721,High,"LINESTRING (8.78201 38.91382, 8.7797 38.9118)"


Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccb89ef00, execution_count=27 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ccb89d0d0, raw_cell="transects[transects.site_id.str.startswith("sar") .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X22sdnNjb2RlLXJlbW90ZQ%3D%3D> result=              site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
sar0001-0000  sar0001          NaN         NaN              NaN          NaN   
sar0001-0001  sar0001          NaN         NaN              NaN          NaN   
sar0001-0002  sar0001          NaN         NaN              NaN          NaN   
sar0001-0003  sar0001          NaN         NaN              N

TypeError: Timer.stop() takes 1 positional argument but 2 were given

In [ ]:
transects.drop_duplicates().to_file("transects_extended.geojson", driver="GeoJSON")

Error in callback <bound method Timer.start of <autotime.Timer object at 0x7f5d25e9bb50>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f5ce107e090, raw_cell="transects.drop_duplicates().to_file("transects_ext.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X23sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: Timer.start() takes 1 positional argument but 2 were given

Error in callback <bound method Timer.stop of <autotime.Timer object at 0x7f5d25e9bb50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5ccaa737d0, execution_count=28 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5ce107e090, raw_cell="transects.drop_duplicates().to_file("transects_ext.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/mnt/c/Users/Daniele/Documents/GitHub/CoastSat/linear_models.ipynb#X23sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Timer.stop() takes 1 positional argument but 2 were given